In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
import statsmodels.formula.api as smf
from fixedeffect.fe import fixedeffect, did, getfe

pd.options.mode.chained_assignment = None

In [2]:
#open data
data = pd.read_csv('pset2.csv')

In [4]:
data['log_fatal_per_cap'] = np.log(data['fatalities']/data['population'])



#some data "cleaning"
data['years_post_prim'] = data['year']
df = pd.DataFrame(columns = data.columns)
data['never_treat'] = 0
for s in sorted(list(set(list(data['state'])))):
    x = 0
    dat = data[data['state']==s]
    for y in list(set(list(data['year']))):
        if dat.iloc[22]['primary']== 0:
            dat['years_post_prim'] = dat['years_post_prim'] - 2004
            #print(dat['years_post_prim'])
            df = df.append(dat)
            break
        if dat.iloc[x]['primary']==1:
            dat['years_post_prim'] = dat['years_post_prim'] - y
            dat['never_treat'] = 1
            df = df.append(dat)
            break
        x = x+1
           
df['post'] = df['years_post_prim']
orig = list(set(list(df['years_post_prim'])))
new = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

df['post'] = df['post'].replace(orig,new)

#assign cohort
df['cohort'] = df['state']
for s in sorted(list(set(list(df['state'])))):
    df['cohort'] = df['cohort'].replace(s, str(df[(df['state']==s)&(df['years_post_prim']==0)]['year'].sum()))
df['cohortc'] = df['cohort']
for y in list(set(list(df['cohort']))):
    if (int(y)!=0)&(int(y) < 1990):
        df['cohortc'] = df['cohortc'].replace(y, '1980')
    if (int(y)>=1990)&(int(y)<2000):
        df['cohortc'] = df['cohortc'].replace(y, '1990')
    if int(y) =='0':
        df['cohortc'] = df['cohortc'].replace(y, '0')
    if int(y)>=2000:
        df['cohortc'] = df['cohortc'].replace(y, '2000')

In [27]:
#3.b

#get state year dummies
data_no_treat = df[df['years_post_prim']<0]
x1 = pd.get_dummies(data_no_treat['state'])
x2 = pd.get_dummies(data_no_treat['year'])
data_no = pd.concat([data_no_treat, x1, x2], axis=1)

x3 = sorted(list(set(list(data_no_treat['state']))))[1:]
x4 = sorted(list(set(list(data_no_treat['year']))))[1:]

#get alphai and betat
vals = sm.OLS(data_no['log_fatal_per_cap'], sm.add_constant(data_no[x3+x4]))
out = vals.fit(cov_type = 'HC0')



x1 = pd.get_dummies(df['state'])
x2 = pd.get_dummies(df['year'])
df4 = pd.concat([df, x1, x2], axis=1)
data2 = df4[df4['years_post_prim']>-1]
#data2 = data2[data2['state']=='CA']


#predict y_i_t
data2['new'] = out.predict(sm.add_constant(data2[x3+x4]))

#get tau (y_true - y_i_t)
data2['tai_i_t'] = data2['log_fatal_per_cap'] - data2['new']

print('ATT')
print(data2[data2['state']=='CA']['tai_i_t'].mean())
print('horizon 0 ATT')
print(data2[(data2['years_post_prim']==0)&(data2['state']=='CA')]['tai_i_t'].mean())


ATT
-0.2708693991058103
horizon 0 ATT
-0.15149702177938917
